In [25]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import LoadJobConfig
from pandas_gbq import to_gbq
import pandas as pd

from google_auth_oauthlib import flow #para hacerlo con la autentifiacion de cliente

In [2]:
scopes=["https://www.googleapis.com/auth/cloud-platform"]
bq_credentials = './credenciales/client_secret_python_bq.json'
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'bigquery-public-data'
table_id = 'ga4_obfuscated_sample_ecommerce.events_20210131'
#table_ref = client.dataset(dataset_id).table(table_id)

In [3]:
credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [4]:
#query de ejemplo
query_ejemplo = """
    SELECT *
    FROM `{}.{}`
    Limit 10
""".format(dataset_id,table_id)

query_ejecutada_ejemplo = client.query(query_ejemplo)
resultado_ejemplo = query_ejecutada_ejemplo.result()

#for row_ejemplo in resultado_ejemplo:
#    print(row_ejemplo)

In [7]:
#crear la tabla si no existe
query1 = """
    CREATE SCHEMA IF NOT EXISTS `{}.{}`(
        columna1 TIPO_DE_DATO1,
        columna2 TIPO_DE_DATO2,
    ...
    )
    PARTITION BY fecha;
""".format(dataset_id,table_id)

In [9]:
#insertar valores en la tabla (¿sustituimos los valores existentes o los añadimods)
query2 = """
    SELECT *
    FROM `{}.{}`
""".format(dataset_id,table_id)

In [ ]:
query_list = [query1, query2]

In [ ]:
for query_i in query_list:
    query_i_ejecuted = client.query(query_i)
    result_query_i = query_i_ejecuted.result()

-----------
#### CREAR TABLA E INSERTAR DATOS

##### a) con query

In [18]:
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_ref = client.dataset(dataset_id).table(table_id)



##### b) a mano

###### 1.A crear la tabla a mano si no existe

In [14]:
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_ref = client.dataset(dataset_id).table(table_id)
try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe en el conjunto de datos {dataset_id}.")
except Exception as e:
    if "Not found" in str(e):
        print(f"La tabla {table_id} no existe en el conjunto de datos {dataset_id}.")
        # Crea la tabla si no existe
        schema = [
            bigquery.SchemaField("cars", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("mpg", "FLOAT", mode="REQUIRED"),
            bigquery.SchemaField("cyl", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("disp", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("hp", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("drat", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("wt", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("qsec", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("vs", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("am", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("gear", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("carb", "INTEGER", mode="NULLABLE")
        ]
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
    else:
        print(f"Error: {e}")

La tabla cars_upload_test no existe en el conjunto de datos chicago_taxi_tips.
Created table curso-bigquery-mide-403114.chicago_taxi_tips.cars_upload_test


###### 1.B crear la tabla leyendo el schema del csv

In [42]:
csv_path = './files/cars.csv'

# Crea una instancia de la tabla en BigQuery (no la crea, solo define el esquema)
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

table_ref = client.dataset(dataset_id).table(table_id)

# Carga una muestra del CSV en un DataFrame de Pandas
sample_df = pd.read_csv(csv_path, sep=",", index_col=False)  # Puedes ajustar el número de filas según tu necesidad

# Crea un diccionario para almacenar el esquema inferido
schema = {}

# Itera sobre las columnas y determina el tipo de datos
for column in sample_df.columns:
    # Infiera el tipo de datos basándose en los tipos de pandas
    if pd.api.types.is_string_dtype(sample_df[column]):
        schema[column] = 'STRING'
    elif pd.api.types.is_numeric_dtype(sample_df[column]):
        schema[column] = 'FLOAT'  # Puedes ajustar según sea necesario
    else:
        schema[column] = 'STRING'  # Puedes ajustar según sea necesario

# iterar este esquema para crear el formato qeu BQ necesita en un listado (ejemplo mas arriba en la creacion a mano del listado)
schema2=[bigquery.SchemaField(keyx, valuex, mode="NULLABLE") for keyx,valuex in schema.items()]

table = bigquery.Table(table_ref, schema=schema2)
table = client.create_table(table)
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

# Configura el esquema de la tabla
#table = bigquery.Table(table_ref, schema=[bigquery.SchemaField(column, schema[column]) for column in schema])

# Imprime el esquema
print(f"Esquema inferido para la tabla {table_id}: {schema}")

Created table curso-bigquery-mide-403114.chicago_taxi_tips.cars_upload_test
Esquema inferido para la tabla cars_upload_test: {'cars': 'STRING', 'mpg': 'FLOAT', 'cyl': 'FLOAT', 'disp': 'FLOAT', 'hp': 'FLOAT', 'drat': 'FLOAT', 'wt': 'FLOAT', 'qsec': 'FLOAT', 'vs': 'FLOAT', 'am': 'FLOAT', 'gear': 'FLOAT', 'carb': 'FLOAT'}


###### 2.A INSERTAR datos del csv en la tabla debajo de los ya existentes

In [43]:
#cargar datos una vez que ya existe (los inserta debajo, si queremos sustituir, habrá que eliminarlos antes)
csv_path ='./files/cars.csv'

# Carga el CSV en un DataFrame de Pandas
df = pd.read_csv(csv_path, sep=",", index_col=False)

# Crea una instancia de la tabla en BigQuery (está hecho arriba, si lo hacemos junto habrá que eliminar las 2 siguientes lineas))
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref)

try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} existe en el conjunto de datos {dataset_id}.")

    job = client.load_table_from_dataframe(df, table)
    job.result()  # Espera a que se complete la carga

    print(f"Datos del CSV cargados en la tabla {table_id} de BigQuery.")
except:
    print(f"La tabla {table_id} NO existe en el conjunto de datos {dataset_id}, hay que crearla")

La tabla cars_upload_test existe en el conjunto de datos chicago_taxi_tips.
Datos del CSV cargados en la tabla cars_upload_test de BigQuery.


###### 2.B REEMPLAZAR datos ya existentes en la tabla por los del csv

In [17]:
#cargar datos una vez que ya existe (los inserta debajo, si queremos sustituir, habrá que eliminarlos antes)
csv_path ='./files/cars.csv'

# Carga el CSV en un DataFrame de Pandas
df = pd.read_csv(csv_path, sep=",", index_col=False)

# Crea una instancia de la tabla en BigQuery (está hecho arriba, si lo hacemos junto habrá que eliminar las 2 siguientes lineas))
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref)

try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} existe en el conjunto de datos {dataset_id}.")
    
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(df, table, job_config=job_config)
    job.result()  # Espera a que se complete la carga

    print(f"Datos del CSV reemplazaron los existentes en la tabla {table_id} de BigQuery.")

except:
    print(f"La tabla {table_id} NO existe en el conjunto de datos {dataset_id}, hay que crearla")

La tabla cars_upload_test existe en el conjunto de datos chicago_taxi_tips.
Datos del CSV reemplazaron los existentes en la tabla cars_upload_test de BigQuery.


##### c) pandas_gbq  -> requiere autenticación

In [10]:
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Ruta al archivo CSV en tu sistema local
csv_path = './files/cars.csv'  # Reemplaza con tu ruta real

# Verifica si la tabla existe en el conjunto de datos
table_ref = client.dataset(dataset_id).table(table_id)
try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe en el conjunto de datos {dataset_id}.")
except Exception as e:
    if "Not found" in str(e):
        print(f"La tabla {table_id} no existe en el conjunto de datos {dataset_id}.")

        # Carga el CSV en un DataFrame de Pandas
        df = pd.read_csv(csv_path, sep=",", index_col=False)

        # Carga el DataFrame en BigQuery
        to_gbq(df, f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

        print(f"La tabla {table_id} ha sido creada y los datos del CSV han sido cargados en BigQuery.")
    else:
        print(f"Error: {e}")

La tabla cars_upload_test ya existe en el conjunto de datos chicago_taxi_tips.
